In [11]:
import insightface
from insightface.app.common import Face
from insightface.model_zoo import model_zoo
from neural_nets import MMFaceFE, MMFaceClassifier, insightface_model, InsightFaceClassifier
import numpy as np
# REQUIRED FOR CUDA TO BE USED
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_subjects = 21

det_model = model_zoo.get_model("../models/buffalo_l/det_10g.onnx")
det_model.prepare(ctx_id=0, input_size=(480, 640), det_thres=0.5)
rec_model = model_zoo.get_model("../models/buffalo_l/w600k_r50.onnx")

# TODO: LOAD THESE MODELS WITH TRAINED WEIGHTS
mmface_model = MMFaceFE().to(device)
mmface_model.eval()

insightface_classifier = InsightFaceClassifier(num_subjects).to(device)
insightface_classifier.eval()

mmface_classifier = MMFaceClassifier().to(device)
mmface_classifier.eval()

def recognise(rgb_input, radar_input):
    with torch.no_grad():
        rgb_emb = insightface_model(rgb_input[..., ::-1], det_model, Face, rec_model)
        radar_emb = mmface_model(radar_input)
    
    return rgb_emb, radar_emb


def classify(rgb_emb, radar_emb):
    with torch.no_grad():
        subject = insightface_classifier(rgb_emb)
        liveness = mmface_classifier(radar_emb)
    
    return subject, liveness

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'do_copy_in_default_stream': '1', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'device_id': '0', 'gpu_external_alloc': '0', 'enable_cuda_graph': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'cudnn_conv_use_max_workspace': '1', 'cudnn_conv1d_pad_to_nc1d': '0', 'tunable_op_enable': '0', 'tunable_op_tuning_enable': '0', 'enable_skip_layer_norm_strict_mode': '0'}, 'CPUExecutionProvider': {}}
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'do_copy_in_default_stream': '1', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'device_id': '0', 'gpu_external_alloc': '0', 'enable_cuda_graph': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo',

In [23]:
import matplotlib.pyplot as plt

DATA_PATH = "../../Soli/soli_realsense/data"

def load_rgb(subject, experiment):
    return np.load(f"{DATA_PATH}/{subject}/{subject}-{experiment}_colour.npy").astype(np.float32)

def recognise_rgb(rgb_input):
    return insightface_model(rgb_input[..., ::-1], det_model, Face, rec_model)

fake_0 = recognise_rgb(load_rgb(90, 0)[0])
fake_0_1 = recognise_rgb(load_rgb(90, 5)[0])
real_0 = recognise_rgb(load_rgb(0, 0)[0])
real_0_1 = recognise_rgb(load_rgb(0, 1)[0])
real_1 = recognise_rgb(load_rgb(1, 0)[0])

In [27]:
score_good = np.clip(np.dot(fake_0_1, fake_0.T), 0., 1.)
score_bad = np.clip(np.dot(fake_0, real_1.T), 0., 1.)

print(score_good, score_bad)

0.3785049319267273 0.0
